# Deep Residual Learning for Image Recognition

# Abstract(논문 개요)
- 심층 신경망은 훈련하기 어렵다.
- 우리는 이전에 사용된 것보다 훨씬 더 깊은 네트워크의 훈련을 쉽게하기 위해서 residual learning 프레임워크를 제시한다.
- 참조되지 않은 함수를 학습하는 것 대신에, layer의 입력값을 참조하여 residual 함수를 학습하는 것으로써 layer를 명확하게 새로 만든다.
- 우리는 종합적인 경험적 증거를 제시한다. 이 residual network는 최적화하는 데 쉽고 꽤 증가된 깊이를 통해 정확도를 얻어내는 것이 가능하다는.
- ImageNet 데이터셋에서, 우리는 VGG 네트워크보다 8배 더 깊은 최대 152개의 층으로 residual 네트워크를 평가하지만 여전히 복잡성은 낮다.
- 이러한 residual 네트워크의 앙상블은 ImageNet 테스트 데이터셋에 대하여 3.57%의 오류를 달성했다.
- 이 결과는 ILSVRC 2015 분류 과제에서 1위를 차지했다.
- 또한 CIFAR-10 데이터에 대한 분석을 100개와 1000개의 layer로 제공한다.
- 표현(representation)의 깊이는 많은 시각적 인식 과제에서 매우 중요하다.
- 단지 극도로 깊은 표현 덕분에, 우리는 COCO 객체 탐지 데이터셋에서 상대적으로 28% 향상하게 되었다.
- Deep residual 네트워크는 ILSVRC & COCO 2015 대회에 제출한 기초이다. 또한 우리는 ImageNet 탐지, ImageNet localization, COCO 탐지, COCO segmentation의 과제에서 1등을 한 이유이다.

> 요점 정리 : residual learning framework는 VGG 네트워크보다 깊지만 복잡성은 낮다. 성능은 더 좋다. residual 네트워크는 최적화가 쉽고 깊어진 깊이를 통해 정확도가 높아졌다.

# 1. Introduction
- deep convolutional neural network는 이미지 분류(image classification)의 일련의 돌파구라고 여겨졌다.
- deep network는 자연스럽게 저/중/고 수준의 feature과 종단간 다층 방식(end-to-end multilayer fashion)으로 통합시켰다. 특징의 "수준"은 쌓인 레이어의 깊이의 수에 의해 강화시킬 수 있다.
- 최근의 증거들에 따르면 네트워크의 깊이가 매우 중요하고 까다로운 ImageNet 데이터셋에 대한 주요 결과는 모두 "매우 깊은" 모델을 활용한다. 깊이는 16 ~ 30이다.
- 다른 사소하지 않은 시각적 인식 작업도 심층 모델(very deep model)의 장점을 많이 받았다.
- 깊이의 중요성에 대한 한 가지 질문이 있다. 더 많은 layer를 쌓는 것만큼 더 나은 네트워크를 배우는 것이 쉬운가?
- 이 질문의 장애물은 처음부터 수렴을 방해하는 vanishing/exploding gradient 라는 악명 높은 문제이다.
- 그러나, 이 문제는 정규화된 초기화(normalized initialization)과 중간 정규화 계층(intermediate normalization layer)에 의해 주로 해결되었다. 이는 수십 개의 계층을 가진 네트워크가 역전파(backpropagation)로 확률적 경사하강법(SGD, stochastic gradient descent)를 위해 수렴을 시작할 수 있게 해준다.
- 더 깊은 네트워크가 수렴을 시작할 수 있을 때, 성능저하(degradation) 문제가 드러났다. 네트워크의 깊이가 증가하면, 정확도가 포화되고(놀랍지 않을 수 있음) 빠르게 저하된다.
- 예상외로 이러한 degradation 문제는 과적합(overfitting)에 의해 발생한 것이 아니고, 적절하게 deep model에 더 많은 layer를 추가하는 것이 훈련 오류를 증가시킨다는 것이 우리의 실험을 통해 철저히 검증되었다.
- 그림 1은 전형적인 예시를 보여준다.
- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FF41Yc%2FbtqDCZoURC2%2FkjRdYZGu7c0IXCbvK11x71%2Fimg.png" width=400>

  - 그림 1. 
  - 왼쪽 : 20층과 56층의 기본(plain) 네트워크를 사용하는 CIFAR-10 데이터셋의 훈련 오류
  - 오른쪽 : 테스트 오류
  - 심층 네트워크는 훈련 오류가 높으므로, 테스트 오류가 발생한다.
  - ImageNet에서 유사한 현상이 그림.4.에 나와있다.
- 훈련 저하도의 저하는 모든 시스템이 비슷하게 최적화하기 쉬운 것은 아니라는 것을 나타낸다.
- 더 얕은 구조(architecture)와 더 많은 layer를 추가하는 더 깊은 대응물(counterpart, 구조)를 고려해보자.
- 더 깊은 모델에 대한 구성별 해결방법이 있다: 추가된 layer는 identity mapping(덧셈 연산), 다른 layer는 학습된 얕은 모델에서 복사
- 이 구성된 해결방법의 존재는 더 깊은 모델이 더 낮은 대응물(counterpart, 모델)보다 더 높은 훈련 오류를 만들어내면 안된다는 것을 나타낸다.
- 하지만, 실험은 현재 사용중인 해결방법은 구성된 해결방법보다 비슷하거나 더 나은 해결방법을 찾을 수 없다고 보여진다. (또는 실현가능한 시간 내에 가능하지 않다.)
- 이 논문에서, 우리는 deep residual learning framework를 소개함으로써 degradation 문제를 해결한다.
- 몇 개의 쌓인 layer 각각이 원하는(desired) 기본(underlying) mapping에 직접 맞기를 바라는 대신에, 이러한 layer가 residual mapping에 명시적으로 맞도록 한다.
- 형식적으로, 원하는 기본 mapping은 $\mathcal{H}(x)$라고 표시하면 쌓인(stacked) 비선형 layer는 $\mathcal{F}(x) := \mathcal{H}(x)-x$의 다른 mapping에 맞도록 한다.
- 원래의 mapping은 $\mathcal{F}(x) + x$로 다시 재구성된다.
- 우리는 원래의 참조되지 않은 mapping을 최적화하는 것보다 residual mapping을 최적화하는 것이 더 쉽다고 가설을 세웠다.
- 극적으로, 만약 identity mapping이 최선이라면, 비선형 layer가 쌓여진 것에 의해 identity mapping을 맞추는 것보다 잔차(residual)을 0으로 밀어내는 것이 더 쉬울 것이다.
- $\mathcal{F}(x) + x$ 식은 "지름길 연결(shortcut connection)"이 있는 feedforward 신경망에 의해 실현될 수 있다. (그림 2)
- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FuhPnp%2FbtqvRaMgqx5%2FAzuZ5zkC2I67gzitMNaeYK%2Fimg.png" width=400>

  - 그림 2.
  - residual learning: a building block
- 지름길 연결(shortcut connection)은 1개 이상의 layer를 건너뛴다.
- 우리의 경우, 지름길 연결은 단순히 identity mapping을 수행하고, identity mapping의 결과는 쌓여진 layer의 출력값에 더해진다.
- identity 지름길 연결은 추가적인 파라미터나 계산 복잡도를 추가하지 않는다.
- 전체 네트워크는 SGD(확률적 경사 하강법)에 의해 end-to-end 훈련이 될 수 있다. 그리고 경사 하강법 알고리즘의 종류를 지정하는 매개변수(solver)를 수정하지 않고도 공통 라이브러리를 사용하여 쉽게 구현할 수 있다.
- 우리는 ImageNet의 종합적인 실험을 제시한다. degradation 문제를 보여주고 우리의 방법을 평가하기 위해서.
- 우리는 1), 2)를 보여준다.
  - 1) 매우 깊은 residual net은 최적화하기에 쉽다.
  - 하지만, 단순히 layer를 쌓기만 한 "일반(plain)" 네트워크는 깊이를 증가시키면 훈련 에러가 더 높아진다.
  - 2) deep residual net은 이전 네트워크보다 훨씬 더 나은 결과를 생성함으로써 매우 깊은 깊이로부터 정확도 향상을 쉽게 얻어낸다.
- 유사한 현상이 CIFAR-10 데이터셋에서도 나타나며, 최적화 어려움과 우리 방법의 효과가 특정 데이터셋과 유사하지 않다는 것을 시사한다.
- 100개 이상의 layer가 있는 이 데이터셋에서 성공적으로 훈련된 모델을 제시하고, 1000개 이상의 layer가 있는 모델을 탐색한다.
- ImageNet 분류 데이터셋에서, 우리는 상당히 깊은 residual net에 의해 우수한 결과를 얻었다.
- 우리의 152개 층의 residual net은 ImageNet에 제시된 가장 깊은 네트워크이지만, 여전히 VGG 네트워크보단 복잡성이 낮다.
- 우리의 앙상블(ensemble)은 ImageNet 테스트 데이터셋에서 3.57%의 top-5 오류를 기록했으며, ILSVRC 2015 분류 대회에서 1등을 했다.
- 또한 매우 깊은 표현(representation)은 다른 인식 작업에서 뛰어난 일반화 성능을 갖고 있고, 더 나아가 ILSVRC와 COCO 2015 대회에서 ImageNet detection, COCO detection, COCO segmentation 분야에서 1등을 했다.
- 이러한 강력한 증거는 보여준다. residual learning 원리가 일반적이고, 다른 시각과 비시각 문제에도 적용되는 것을 기대할 수 있다고.

> 요점정리 : 

# 2. Related Work(관련 작업)

# 2.1. Residual Representations(잔여 표현)
- 이미지 인식에서, VLAD는 사전(dictionary)에 대하여 residual vector로 인코딩하는 표현이다. 그리고 Fisher Vector는 VLAD의 확률적인 버전으로 만들 수 있다.
- 둘 다(VLAD, Fisher Vector) 영상 검색(image retrieval) 및 분류(classification)를 위한 강력한 얕은 표현이다.
- 벡터 양자화(vector quantization)에서, residual vector를 인코딩하는 것은 원래의 vector를 인코딩하는 것보다 더 효과적이라고 보여진다.
- 저수준의 vision, computer graphics에서, 편미분 방정식(PDEs, Partial Differential Equations)을 풀기 위해, 널리 사용되는 Multigrid 방법은 시스템을 여러 척도의 하위 문제로 재구성한다. 여기서 각 하위 문제는 결이 거친것(coarser)과 섬세한(finer) 스케일 사이에서 residual 해결방법에 책임이 있다.
- Multigrid 방법의 대안은 계층 기반의 전제 조건 공식(hierarchical basis preconditioning)이다. 이 방법은 두 개의 스케일(척도) 사이에서 residual vector를 나타내는 변수에 의존한다.
- 이러한 solver는 해결방법의 나머지 특성(residual nature)를 잘 인식하지 못하는 standard solver보다는 훨씬 더 빨리 수렴한다.
- 이러한 방법은 좋은 재구성 또는 전제 조건 공식이 최적화를 단순화할 수 있다고 말하고 있다.

# 2.2. Shortcut Connections(지름길 연결)
- 지름길 연결에 대한 훈련과 이론은 오랜시간동안 연구되어왔다.
- 다중 계층 퍼셉트론(MLP, multi-layer perceptron)에 대한 초기 훈련은 네트워크 입력에서 출력으로 연결된 선형 계층을 추가하는 것이다.
- vanishing/exploding gradient 문제를 해결하기 위해서 몇 개의 중간 layer가 보조적인 분류기(auxiliary classifier)에 직접 연결되었다.
- 일부 논문은 layer 응답(response), 기울기(gradient), 전파된 오차(propagated error)를 중심화하는 방법을 제안했다. 그리고 그것들은 지름길 연결에 의해 구현된다.
- "inception"(GoogleNet) layer는 지름길 branch와 몇 개의 더 깊은 branch로 구성된다.
- 우리의 작업과 동시에, "highway networks"는 gating function과 함께 지름길 연결을 제시한다.
- 이러한 gate는 자료에 종속되어있고(data-dependent) 파라미터로부터 자유로운(parameter-free) 우리의 identity shortcut과는 반대로 파라미터가 있다.
- 게이트된 지름길(gated shortcut)은 0에 가깝게 "닫히게" 되면, highway network의 layer는 non-residual function을 나타낸다.
- 반대로, 우리의 공식은 항상 residual function을 학습한다. 우리의 identity 지름길은 항상 닫혀있고, 모든 정보는 합습할 추가된 residual function과 함께 전달된다.
- 또한 highway network는 상당히 깊은 깊이(예, 100개 이상의 layer)로 정확도가 향상되지 않음을 증명한다.

# 3. Deep Residual Learning

# 3.1. Residual Learning(잔여 학습)
- $\mathcal{H}(x)$ : 몇 개의 쌓여진 layer(반드시 전체 layer가 아니어도 됨)에 맞는 기본 mapping
- $x$ : 이러한 layer 중 1번째 layer에 대한 입력
- 여러 비선형 계층이 복잡한 함수에 점근적으로 근사할 수 있다고 가정하면, 잔차함수에 점근적으로(asymptotically) 근사할 수 있다는 가설과 동일하다. 여기서 잔차 함수는 $\mathcal{H}(x) - x$(입력과 출력이 동일한 차원이라고 가정)
- 따라서 쌓인 layer가 $\mathcal{H}(x)$에 근접할 것으로 기대하기보다는, 이 layer가 잔차함수 $\mathcal{F}(x) := \mathcal{H}(x) - x$에 근접하도록 명시적으로 한다.
- 따라서 원래 함수는 $\mathcal{F}(x) + x$가 된다.
- 두 형식 모두 원하는 기능을 점근적으로 근사할 수 있어야 하지만 가설에 따라 학습의 쉬운 정도는 다를 수 있다.
- 이러한 재구성은 degradation 문제에 대한 직관에 어긋나는(conuterintuitive) 현상(phenomena)에 의해 발생된다.(motivated) (그림 1, 왼쪽 그림)
- introduction에서 이야기한 것 처럼, 추가된 layer를 identity mapping으로 구성할 수 있을 경우, 더 깊은 모델은 얕은 대응물(counterpart, 모델)보다 더 크지 않은 훈련 오류를 가져야만 한다.
- degradation 문제는 최적화 방법(solver)이 여러 비선형 layer에 의해 identity mapping을 근사하는데 어려움이 있을 수 있다고 시사한다. 
- residual learning을 재구성하여, identity mapping이 최적화된경우, 최적화 방법(solver)는 단순히 identity mapping에 접근하기 위해서 많은 비선형 layer의 가중치들을 0쪽으로 만들 수 있다.
- 실제 경우에는, identity mapping이 최적화될 것 같진 않지만, 우리의 재구성은 아마도 문제의 전제조건(precondition)을 도와줄 수 있을 것이다.
- 만약 최적화 함수가 zero mapping보다 identity mapping에 더 가깝다고 한다면, 최적화 방법(solver)가 새로운 함수를 학습하는 것보단 identity mapping을 참조하여 작은 변화(perturbation)을 찾는 것이 더 쉬울 것이다.
- 우리는 실험을 통해 학습된 residual 함수가 일반적으로 작은 응답(response)를 가지고 있음을 보여줌으로써, identity mapping이 합리적인 전제조건을 제공한다고 시사한다.

# 3.2. Identity Mapping by Shortcuts
- 우리는 모든 building layer에 residual learning을 채택한다.
- building block은 그림 2에 있다.
- 공식적으로, 이 논문에서 우리는 $y = \mathcal{F}(x, \{\mathcal{W}_i\}) + x$ 로 정의된 쌓인 블록을 고려한다.
  - x : 고려되는 layer의 입력
  - y : 고려되는 layer의 출력
- 함수 $\mathcal{F}(x, \{\mathcal{W}_i\})$는 학습할 residual mapping이다.
- 예) 그림 2의 2번째 layer를 갖고있는 것에서, $\mathcal{F} = \mathcal{W}_{2}\sigma(\mathcal{W}_{1}x)$ 에서 $\sigma$는 ReLU를 나타내고 표기법을 단순화하기 위해 편향을 생략했다.
- 작업(operation) $\mathcal{F} + x$는 지름길 연결 및 요소별 덧셈(element-wise addition)으로 수행된다.
- 우리는 덧셈을 수행한 후 2번째 비선형성을 채택한다. (즉, 그림2의 $\sigma(y)$)
- $y = \mathcal{F}(x, \{\mathcal{W}_i\}) + x$ 이 공식의 지름길 연결은 추가 매개변수나 계산 복잡도를 도입하지 않는다.
- 이것은 실제로 매력적일뿐만 아니라 일반 네트워크와 residual network간의 비교에서도 중요하다.
- 무시할 수 있는 요소별 덧셈을 제외하고 동일한 수의 파라미터, 깊이, 너비, 계산 비용(computational cost)을 동시에 갖는 일반(plain)/잔차(residual) 네트워크를 공정하게 비교할 수 있다.
- x와 $\mathcal{F}$의 차원 수는 $y = \mathcal{F}(x, \{\mathcal{W}_i\}) + x$ 이 공식에서 동일해야 한다.
- 그렇지 않은 경우(예, 입력/출력 채널을 변경할 때) 지름길 연결로 선형 투영(linear projection) $\mathcal{W}_s$을 수행하여 차원수를 일치시킬 수 있다. : $y = \mathcal{F}(x, \{\mathcal{W}_i\}) + \mathcal{W}_{s}x$
- 우리는 식 $y = \mathcal{F}(x, \{\mathcal{W}_i\}) + x$에서 정사각형 행렬 $\mathcal{W}_x$를 사용할 수도 있다.
- 하지만 우리는 identity mapping이 degradation 문제를 해결하기에 충분하고 경제적이므로, $\mathcal{W}_s$는 차원을 일치시킬 때만 사용된다는 것을 실험을 통해 보여줄 것이다.
- residual function $\mathcal{F}$은 유연하다.(flexible)
- 이 논문의 실험은 2개 또는 3개의 layer(그림 5)를 갖는 함수 $\mathcal{F}$를 포함하지만 더 많은 layer가 가능하다.
- 하지만, $\mathcal{F}$가 single layer만 갖고 있는 경우, 식 $y = \mathcal{F}(x, \{\mathcal{W}_i\}) + x$ 은 선형 layer와 유사하다. : $y = \mathcal{W}_{1}x + x$, 장점은 관찰되지 않았다.
- 우리는 또한 위의 표기법은 단순함(simplicity)을 위해 fully-connected layer에 대한 것이지만, convolutional layer에도 적용할 수 있다.
- 함수 $\mathcal{F}(x, \{\mathcal{W}_i\})$ 는 다중 컨볼루젼 레이어를 나타낼 수 있다.
- 요소별 덧셈은 채널별로 2개의 feature map에서 수행된다.


# 3.3. Network Architectures
- 다양한 일반(plain)/잔여(residual) 네트워크를 테스트했더니 공통적인 현상이 관찰되었다.
- 토론을 위한 사례를 제공하기 위해서, ImageNet의 2가지 모델을 다음과 같이 설명한다.
1. 일반 네트워크(Plain Network)
- 우리의 기본 네트워크의 기준치(plain baseline)(그림 3의 가운데 그림)는 주로 VGG 네트워크(그림3의 왼쪽 그림)의 철학에서 영감을 받았다.
- <img src="https://sike6054.github.io/blog/images/ResNet,%20Fig.3(removed).png" width=700>

- 그림 3.
  - ImageNet용 네트워크 구조의 예시.
  - 왼쪽 : VGG-19 모델(196억 FLOPS)을 참조로 사용
  - 중간 : 32개의 파라미터 layer(36억 FLOPS)가 있는 일반 네트워크
  - 오른쪽 : 34개의 파라미터 layer(36억 FLOPS)가 있는 residual 네트워크.
  - 점선 지름길은 차원수를 증가시킨다.
  - 표 1 : 자세한 내용과 기타 변수들에 대한 설명
- 컨볼루젼 레이어는 대부분 3x3 필터를 갖고 있고 2가지 간단한 디자인 규칙을 따른다.
  - 1) 동일한 출력 feature map의 크기를 위해, layer들은 동일한 필터의 갯수를 가지고 있다.
  - 2) 만약 feature map의 크기가 절반으로 줄어든다면, layer당 시간 복잡도(time complexity)를 보존하기 위해서 필터의 수가 2배로 늘어난다.
- stride=2인 컨볼루젼 layer에 직접적으로 downsampling을 수행한다.
- 네트워크는 global average pooling layer와 softmax가 있는 1000-way fully-connected layer로 끝난다.
- 가중치 layer의 총 갯수 : 34개(그림 3, 중간 그림)
- ResNet 모델이 VGG 네트워크보다 필터의 갯수가 적고 복잡성이 낮다는 점에서 주목할만한 가치가 있다. (그림 3, 왼쪽 그림)
- 34개의 layer baseline은 36억 FLOPS를 갖는다. (multiply-adds) VGG-19 모델(196억 FLOPS)의 18%에 불과하다.
  - FLOPS 가 클수록 계산하는 데 시간이 오래 걸린다는 것으로 안 좋은 것이다. 따라서, VGG-19 모델보다 ResNet 모델을 사용했을 때 계산 속도가 훨씬 더 빠르다는 것이다.

2. 잔여 네트워크(Residual Network)
- 위의 일반(plain) 네트워크를 기반으로 지름길 연결(그림 3, 오른쪽 그림)을 삽입하여 네트워크를 counterpart residual version으로 바꾼다.
- identity 지름길(식 $y = \mathcal{F}(x, \{\mathcal{W}_i\}) + x$)은 입력과 출력값이 동일한 차원수일 때(그림 3, 실선 지름길) 직접 사용할 수 있다.
- 차원수가 증가하면(그림 3, 점선 지름길) 2가지 옵션을 고려한다.
  - (A) 지름길은 여전히 identity mapping을 수행한다. 차원수 증가를 위해 나머지는 0으로 채워진다.
  - 이 방법은 매개변수를 추가로 도입하지 않는다.
  - (B) 식 $y = \mathcal{F}(x, \{\mathcal{W}_i\}) + \mathcal{W}_{s}$의 투영 지름길(projection shortcut)은 차원수를 맞춰주는 데 사용된다. (1x1 컨볼루젼으로 수행된다.)
- 두 옵션 모두, 지름길이 2가지 크기의 feature map을 가로지르면, stride=2로 수행된다.

# 3.4. Implementation(구현)
- 이미지는 크기 증가(size augmentation)을 위해서 무작위로 샘플링된 짧은 면(shorter side)으로 크기가 조정된다.
- 224x224 크롭은 이미지나 수평 뒤집기(horizontal flip, 좌우반전)에서 무작위로 샘플링되며 픽셀당 평균을 뺀다.(subtract)
- 표준 색상 증가(standard color augmentation) 기술이 사용된다.
- 우리는 각각의 컨볼루젼 이후나 활성화 함수 직전에 배치 정규화 방법(batch normalization)을 사용한다.
- 가중치를 초기화하고 모든 일반(plain)/잔여(residual) 네트워크를 처음부터 훈련한다.
- 우리는 미니배치(mini-batch) 크기가 256인 SGD(확률적 경사 하강법)을 사용한다.
- 학습률은 0.1부터 시작하여 오류가 안정기에 접어들었을 때(plateau), 모델은 최대 60x104 iteration으로 학습된다.
- 우리는 weight decay 0.0001과 momentum 0.9를 사용한다.
- 우리는 논문 " S. Ioffe and C. Szegedy. Batch normalization: Accelerating deep network training by reducing internal covariate shift. In ICML, 2015."의 관행에 따라 dropout을 사용하지 않겠다.
- 테스트 과정에서, 비교 연구를 위해 우리는 표준 10-crop 테스트 과정을 채택한다.
- 최고의 결과를 얻기 위해서, 우리는 fully onvolutional form 형식을 채택하고, 여러 가지의 척도에서 점수의 평균을 구한다. (짧은 면이 224, 256, 384, 480, 640이 되도록 이미지 크기가 조정)

# 4. Experiments

# 4.1. ImageNet Classification
- 데이터셋 : ImageNet 2012 분류 데이터셋
- 클래스의 갯수 : 1000개
- 훈련 데이터 : 128만개
- 검증 데이터 : 5만개
- 테스트 데이터 : 10만개
- 상위 1 ~5번째의 오류율 평가

# 4.1.1. Plain Networks(일반 네트워크)
- 18-layer, 34-layer plain 네트워크 평가
- 34-layer plain net은 그림 3, 가운데 그림이다.
- 18-layer plain net도 비슷한 형태이다.
- 자세한 구조는 표 1을 참조.
- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FGoNnQ%2FbtqDXtQoZXe%2FqA3MJuFg7Ns2PLGkaoJKx0%2Fimg.png" width=1000>

  - 표 1.
  - ImageNet을 위한 구조.
  - Building block은 괄호(bracket)안에 표시되며,(그림 5) 블록 수가 쌓여있다.
  - downsampling은 conv3_1, conv4_1, conv5_1에 의해 stride=2로 수행된다.
- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcLBpBk%2FbtqDVUVtWqf%2FwbpEWR20zFrUC279DjAT40%2Fimg.png" width=500>

  - 표 2.
  - ImageNet 검증에 대한 top-1 오류(%단위, 10-crop testing)
  - 여기서 ResNet은 일반 네트워크와는 다르게 추가 매개변수가 없다.
  - 더 얕은 18-layer plain network보다 더 깊은 34-layer plain network가 검증 오차가 더 높다.
  - 그 이유를 밝히기 위해서 그림 4, 왼쪽 그림에서 훈련 절차 중의 훈련/검증 오류를 비교한다.
- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcfjTsP%2FbtqDVvBDzLz%2FAC6alGVGxgN8hdRLhLbRE1%2Fimg.png" width=1000>

  - 그림 4.
  - ImageNet 훈련.
  - 얇은 곡선 : 훈련 오류(training error)
  - 굵은 곡선 : 중앙 crop의 검증 오류(validation error)
  - 왼쪽 : 18-layer, 34-layer의 plain network
  - 오른쪽 : 18-layer, 34-layer의 residual network(ResNet)
  - 이 그래프에서 residual network는 plain network와는 다르게 추가 모수가 없다.
  - degradation 문제가 관찰되었다.
  - 18-layer plain network의 해공간(solution space)가 34-layer plain network의 해공간의 부분공간(subspace)라고 할지라도, 34-layer plain network는 전체 훈련 과정에서 더 높은 훈련 오류를 가진다.
- 이러한 문제에 대해 최적화 어려운 문제(optimization difficulty)는 vanishing gradient 문제 때문에 발생한 건 아니라고 주장한다.
- 이러한 plain network는 배치 정규화(BN)로 훈련되어, 순방향 전파 신호(forward propagated signal)가 0이 아닌 분산을 반드시 갖도록 만든다.
- 또한 역전파된 기울기(backward propagated gradient)가 배치 정규화(BN)와 함께 healthy norm을 나타내는지 입증했다.
- 따라서 순전파 신호도 역전파 신호도 사라지지 않는다.(vanish)
- 실제로, 34-layer plain net은 경쟁적인 정확도를(표 3) 달성했다. solver가 어느 정도 작동함을 시사했다.
- deep plain net는 아마도 기하급수적으로 낮게 수렴할지도 모른다. 이는 훈련 오류의 감소에 영향을 미친다.
- 이러한 최적화 문제의 원인은 향후 연구될 것이다.

# 4.1.2. Residual Networks(잔여 네트워크)
- 다음으로 18-layer, 34-layer residual net(ResNet)을 평가하자.
- 기준 아키텍쳐(baseline architecture)는 위의 plain net와 동일하다. 그림 3, 오른쪽 그림과 같이 각각의 3x3 필터쌍에 지름길 연결이 추가될 것으로 예상한다.
- 1번째 비교(표2, 그림 4, 오른쪽 그림)에서 모든 지름길과 zero-padding을 차원수를 늘리기 위해서(option A) identity mapping을 사용한다.
- 따라서 그들은 plain 네트워크에 비해 추가 파라미터가 없다.
- 표2, 그림 4로부터 3가지 주요 관찰 결과가 있다.
  1. 상황이 residual learning에서 뒤바뀌었다. 34-layer ResNet이 18-layer ResNet보다 낫다. (2.8%)
  - 더 중요한 것은, 34-layer ResNet이 상당히 낮은 훈련 오류율을 보여주고, 검증 데이터로 일반화 할 수 있다는 것이다.
  - 이것은 degradation 문제가 잘 해결되었음을 나타낸다. 그리고 증가된 깊이에서 정확도를 얻을 수 있게 되었다.
  2. plain ResNet과 비교하여 34-layer ResNet은 top-1 오류를 3.5% 감소시켰다. (표 2). 훈련 오류가 성공적으로 감소되었기 때문이다. (그림 3, 오른쪽 그림 vs 왼쪽 그림)
  - 이 비교는 매우 깊은 시스템에서의 residual learning의 효과를 확인한다.
  3. 18-layer plain/residual net은 비교적으로 정확하지만(표 2), 18-layer ResNet은 더 빠르게 수렴한다.(그림 4, 오른쪽 vs 왼쪽)
  - 네트워크가 "너무 깊지 않은" 경우(여기서는 18-layer), 현재의 SGD solver는 plain net에 대해서 좋은 해결책을 찾을 수 있다.
  - 이 경우, ResNet은 초기 단계에서 더 빠른 수렴을 제공하여 최적화를 쉽게 만들어준다.

# 4.1.3. Identity vs Projection Shortcuts
- 우리는 파라미터가 없는, identity 지름길(shortcut)이 훈련에 도움이 된다는 것을 보여주고 있다.
- 다음으로 우리는 투영 지름길(projection shortcut)을 조사한다.
- <img src="https://t1.daumcdn.net/cfile/tistory/99D71B375B13DDAE33" width=400>

  - 표 3.
  - ImageNet 검증에서의 오류율(%, 10-crop testing)
  - VGG-16은 우리의 테스트를 기반으로 한다.
  - ResNet-50/101/152는 차원수 중가를 위해 오직 projection만 사용하는 옵션 B이다.
- 표3, 우리는 3가지 옵션이 있다.
  - (A) zero-padding shortcut은 차원수를 늘리는 데 사용된다. 그리고 모든 shortcut은 파라미터가 없다.(표 2와 그림 4, 오른쪽 그림과 동일하게)
  - (B) projection shortcut은 차원수를 늘리는 데 사용된다. 그리고 모든 shortcut은 identity하다.
  - (C) 모든 shortcut은 projection이다.
- 표3은 3가지 옵션이 모두 plain 네트워크보다 훨씬 낫다는 것을 보여준다. 옵션 B는 옵션 A보다 약간 낫다. 이는 옵션 A의 제로패딩된(zero-padded) 차원에는 사실 residual learning이 없기 때문이라고 우리는 주장한다.
- 옵션 C는 옵션 B보다 약간 낫다. 그리고 우리는 이것을 많은 (13개의) projection shortcut에 의해 도입된 추가된 매개변수 때문이라고 생각한다.
- 하지만, 옵션 A/B/C간의 작은 차이는 projection shortcut이 degradation 문제를 해결하는데 필수적이지 않다는 것을 나타낸다.
- 따라서 이 논문의 나머지 부분에서는 메모리/시간 복잡성(memory/time complexity)와 모델의 크기를 줄이기 위해 옵션 C를 사용하지 않겠다.
- identity shortcut은 아래에 소개된 병목 구조(bottleneck architecture)의 복잡성(complexity)을 증가시키지 않는데 특히 중요하다.


# 4.1.4. Deeper Bottleneck Architectures(더 깊은 병목 구조)
- 다음으로 ImageNet에 대한 더 깊은 네트워크에 대해 설명한다.
- 우리가 감당이 되는 훈련 시간에 대한 걱정 때문에, 우리는 building block을 병목 설계(bottleneck design)로 수정한다.
- 각 residual function F에서, 우리는 2개(그림 5) 대신에 3개의 layer를 쌓는 것을 사용한다.
- 이 3개의 layer는 1x1, 3x3, 1x1 컨볼루젼이다. 1x1 layer는 차원 수를 증가시키고(복원시키고) 감소시키는데 책임이 있다. 3x3 layer의 입력/출력의 차원수를 줄인 bottleneck으로 둔다.
- 그림 5는 두 디자인의 시간 복잡도가 비슷하다는 것을 보여준다.
- <img src="https://sike6054.github.io/blog/images/ResNet,%20Fig.5(removed).png" width=600>
  
  - 그림 5.
  - ImageNet에 대한 더 깊은 잔차 함수 F
  - 왼쪽 : ResNet34에 대해 그림 3과 같은 building block(56x56 feature map)
  - 오른쪽 : ResNet-50/101/152의 "병목현상(bottleneck)" building block

- 매개변수가 없는 identity shortcut은 특히 병목 구조를 위해 중요하다.
- 그림 5, 오른쪽 그림의 identity shortcut은 projection으로 대체가 된다면, 시간 복잡도와 모델의 크기가 2배가 됨을 보여준다. shortcut이 2개의 높은 차원의 끝부분에 연결되어있기 때문이다.
- identity shortcut은 bottleneck design을 위해 더 효과적인 모델로 이어진다.

# 4.1.5. 50-layer ResNet
- 우리는 34-layer net의 2-layer block을 3-layer bottleneck block으로 교체한다. 그 결과 50-layer ResNet(표 1)가 되었다.
- 우리는 차원수를 늘리기 위해서 옵션 B를 사용한다.
- 이 모델은 38억 FLOPS을 가진다.

# 4.1.6. 101-layer and 152-layer ResNets
- 더 많은 3-layer block을 사용함으로써 101-layer과 152-layer ResNet을 구성한다.(표 1)
- 놀랍게도, 깊이가 엄청 증가했지만, 152-layer ResNet(113억 FLOPS)은 여전히 VGG-16/19 네트워크(153/196억 FLOPS)보다 복잡성이 낮다.
- 50/101/152-layer ResNet은 34-layer ResNet보다 상당한 차이(margin)로 더 정확하다.(표 3, 4)
- <img src="https://sike6054.github.io/blog/images/ResNet,%20Table.4(removed).png" width=500>
  
  - 표 4.
  - ImageNet 검증 데이터셋(테스트 세트에 보고된 † 제외)에 대한 단일 모델(single-model)의 결과의 오류율(%)
- 우리는 degradation 문제를 관찰하지 않는다. 그래서 상당히 증가된 깊이로부터 상당한 정확도가 향상되고 있다.
- 깊이의 장점은 모든 평가 지표에서 증명되었다.(witness)(표3, 4)

# 4.1.8. Comparisons with State-of-the-art Methods(최신의 방법들과 비교)
- 표 4에서 우리는 이전의 최고의 단일 모델(single-model)의 결과와 비교한다.
- 우리의 기준(baseline) 34-layer ResNet은 매우 경쟁력 있는 정확도를 달성했다.
- 우리의 152-layer ResNet은 4.49%의 단일 모델(single model) top-5 검증 오류가 있다.
- 이 단일 모델의 결과는 이전의 모든 앙상블(ensemble)의 결과보다 더 좋은 결과가 나왔다.
- 우리는 서로 다른 6가지 모델을 결합하여 앙상블을 형성한다. (제출 당시 2개의 152-layer 모델만 사용했음)
- 이로 인해 테스트 데이터셋에서 3.57% top-5 오류가 나왔다. (표 5)
- <img src="https://sike6054.github.io/blog/images/ResNet,%20Table.5(removed).png" width=500>
  - 표 5.
  - 앙상블의 오류율(%)
  - 상위 5개(top-5)의 오류는 ImageNet의 테스트 데이터셋에 있고 테스트 서버에서 보고한다.

- 이 출품작(entry)은 ILSVRC 2015에서 1위를 했다.

# 4.2. CIFAR-10 and Analysis
- 데이터셋 : CIFAR-10
- 훈련 이미지 갯수 : 5만개
- 클래스의 갯수 : 10개
- 테스트 이미지 갯수 : 1만개
- 훈련 데이터셋으로 훈련하고 테스트 데이터셋에서 평가된 실험을 보여주겠다.
- 우리는 매우 깊은 네트워크의 동작에 초점을 맞췄다.하지만, 최신의 결과를 추진하는 것은 아니다. 그러므로, 의도적으로 다음과 같이 간단한 구조를 사용한다.
- 일반(plain)/잔여(residual) 구조는 그림 3(가운데/오른쪽)의 형식을 따른다.
- 네트워크의 입력은 32x32 이미지이다. 각 픽셀당 평균을 뺀.
- 첫 번째 layer는 3x3 컨볼루젼이다.
- 그 다음엔 크기가 {32, 16, 8}인 feature map에 3x3 컨볼루젼이 있는 6n layer stack을 사용하고, 각각의 feature map의 크기에 대해 2n layer를 사용한다.
- 필터의 수는 {16, 32, 64}이다.
- subsampling은 stride=2인 컨볼루젼에 의해 수행된다.
- 네트워크는 global average pooling, 10-way fully-connected layer, softmax로 끝난다.
- 총 6n+2개의 누적 가중치 레이어(stacked weighted layer)가 있다.
- 다음 표에는 구조가 요약되어있다.
- <img src="https://www.programmersought.com/images/678/9ecee93a3fcbcfab3db7427a2ca33d36.png" width=500>

- shortcut 연결을 사용하면, 그들은 3x3 layer 쌍(총 3n shortcut)에 연결된다.
- 이 데이터셋에서 우리는 모든 경우(즉, 옵션 A)에서 identity shortcut을 사용한다. 그러므로 residual model은 plain 모델과 정확히 동일한 깊이, 너비, 매개변수의 수를 갖는다.
- 우리는 weight decay 0.0001과 momentum 0.9를 사용하고, 논문 "K. He, X. Zhang, S. Ren, and J. Sun. Delving deep into rectifiers: Surpassing human-level performance on imagenet classification. In ICCV, 2015."의 가중치 초기화 방법을 채택하지만 배치 정규화는 없다.
- 이러한 모델은 2개의 GPU에서 128개의 미니 배치 크기로 학습된다.
- 학습률 0.1로 시작하여 32000번째, 48000번째 반복(iteration)에서 10으로 나누고 45000/5000 train/val split에서 결정되는 64000 반복(iteration)에서 훈련을 종료시킨다.
- 우리는 훈련을 위해 data augmentation을 한다. : 각 면(side)에 4pixel이 패딩되고(pad), 32x32 crop은 랜덤으로 패딩된 이미지(padded image)나 수평적으로 반전된 이미지(horizontal flip)에서 무작위로 샘플링된다.
- 테스트를 위해, 우리는 원본 32x32 이미지의 단일 보기(single view)만을 평가한다.
- 우리는 n = {3, 5, 7, 9}를 비교하여, 20, 32, 44, 56-layer 네트워크로 이어진다.
- 그림 6, 왼쪽 그림이 plain 네트워크의 행동을 보여준다.
- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F5zVfe%2FbtqDVuCLG1a%2FJ35Ojj9NmARfEKgsyPop30%2Fimg.png" width=800>

  - 그림 6.
  - CIFAR-10 데이터셋에 대한 훈련
  - 파선(점선, dashed line) : 훈련 오류
  - 굵은 선 : 테스트 오류
  - 왼쪽 : plain network. plain-110의 오류는 60%이상이고 표시하지 않았다.
  - 가운데 : ResNet
  - 오른쪽 : 110, 1202개의 layer가 있는 ResNet
- deep plain net는 증가된 깊이로 인해 고통받는다. 그리고, 더 깊이 갈 때 더 높은 훈련 오류가 보인다.
- 이러한 현상은 ImageNet(그림 4, 왼쪽 그림) 및 MNIST에서 유사하게 나타난다. 이는 이러한 최적화 어려움(optimization difficulty)이 근본적인 문제임을 시사한다.
- 그림 6, 가운데 그림은 ResNet의 행동을 보여준다.
- 또한 ImageNet 경우(그림 4, 오른쪽 그림)와 비슷하게, 우리의 ResNet은 최적화 어려움(optimization difficulty)을 극복하고 깊이가 증가할 때 정확도 향상을 보여준다.
- 110 layer ResNet으로 이어지는 n=18을 더 탐구해보자.
- 이 경우에, 0.1의 초기 학습률(initial learning rate)이 수렴을 시작하기에 너무 크다는 것을 발견했다.
- 따라서 0.01을 사용하여 훈련 오류가 80% 미만(약 400 반복(iteration))이 될때까지 훈련을 워밍업(warm up)한 다음 0.1로 돌아가서 훈련을 계속한다.
- 나머지 학습 과정(learning schedule)은 이전과 같다.
- 이 110-layer network는 잘 수렴된다.(그림 6, 가운데 그림)
- FitNet, Highway(표 6)와 같은 다른 deep, thin 네트워크보다 매개 변수가 적지면 최신의 결과(6.43%, 표 6)에 속한다.
- <img src="https://t1.daumcdn.net/cfile/tistory/99ED3B465CB8A11F09" width=500>

  - 표 6.
  - CIFAR-10 테스트 데이터셋의 분류 오류(classification error)
  - 모든 방법에는 데이터 증가(data augmentation)이 있다.
  - ResNet-110의 경우, 5번 실행하고 "best(mean±std)"를 표시한다.

# 4.2.1. Analysis of Layer Responses.
- 그림 7은 계층 응답(layer response)의 표준편차(std, standard deviation)을 보여준다.
- <img src="https://t1.daumcdn.net/cfile/tistory/99FABD335B13DE1731" width=500>
  
  - 그림 7.
  - CIFAR-10에 대한 계층 응답의 표준 편차
  - 응답은 배치 정규화(BN) 이후와 비선형(nonlinearity)이전의 각 3x3 레이어의 출력이다.
  - 위 그림 : 레이어가 원래 순서대로 표시된다.
  - 아래 그림 : 응답의 순위가 내림차순으로 지정된다.

---
---
[ResNet 논문 추가 개념 정리](https://github.com/yoonchaiyoung/ML-DL_Study/blob/master/%EB%AA%A8%EB%A5%B4%EB%8A%94%20%EA%B0%9C%EB%85%90%20%EA%B2%80%EC%83%89%ED%95%98%EC%97%AC%20%EC%A0%95%EB%A6%AC.ipynb)
